# Autograd
The autograd package provides automatic differentiation 
 for all operations on Tensors

In [35]:
import torch
#requires_grad =True-> tracks all operations on the operator
x=torch.randn(3,requires_grad=True)
y=x+2
# y was created as a result of an operation, so it has a grad_fn attribute.
# grad_fn: references a Function that has created the Tensor
print(x)# created by the user-> grad_fn is None
print(y)
print(y.grad_fn)

tensor([0.3134, 0.5469, 0.4309], requires_grad=True)
tensor([2.3134, 2.5469, 2.4309], grad_fn=<AddBackward0>)


In [36]:
#do more operations on y
z=y*y*3
print(z)
z=z.mean()
print(z)

tensor([16.0550, 19.4599, 17.7282], grad_fn=<MulBackward0>)
tensor(17.7477, grad_fn=<MeanBackward0>)


Let's compute the gradients with backpropagation.
When we finish our computation we can call backward() and have all the gradients computed automatically. The gradient for this tensor will be accumulated into grad attribute.
It is the partial derivate of the function w.r.t. the tensor

In [37]:
z.backward()
print(x.grad)

tensor([4.6267, 5.0938, 4.8618])


Generally speaking, torch.autograd is an engine for computing vector-Jacobian product.
It computes partial derivates while applying the chain rule.
 
 Model with non-scalar output:
 If a Tensor is non-scalar (more than 1 elements), we need to specify arguments for backward() .
 specify a gradient argument that is a tensor of matching shape.
 needed for vector-Jacobian product

In [38]:
x=torch.randn(3,requires_grad=True)
y=x*2
for _ in range(10):
    y*=2
print(y)
print(y.shape)

tensor([ -662.0009,  2165.6428, -1005.0162], grad_fn=<MulBackward0>)
torch.Size([3])


In [39]:
v=torch.tensor([0.1,1.0,0.0001],dtype=torch.float32)
y.backward(v)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])



##### Stop a tensor from tracking history:
 For example during our training loop when we want to update our weights
 then this update operation should not be part of the gradient computation
- x.requires_grad_(False)
 - x.detach()
- wrap in 'with torch.no_grad():'

.requires_grad_(...) changes an existing flag in-place.

In [40]:
a=torch.randn(2,2)
print(a.requires_grad)
b=((a*3)/(a-1))
print(b.grad_fn)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)


False
None
True


##### .detach(): get a new Tensor with the same content but no gradient computation:


In [41]:
a=torch.randn(2,2,requires_grad=True)
print(a.requires_grad)
b=a.detach()
print(b.requires_grad)

True
False


#### wrap in 'with torch.no_grad():'

In [42]:
a=torch.randn(2,2,requires_grad=True)
print(a.requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)

True
False


backward() accumulates the gradient for this tensor into .grad attribute.
!!! We need to be careful during optimization !!!
Use .zero_() to empty the gradients before a new optimization step!

In [45]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    # just a dummy example
    model_output = (weights*3).sum()
    model_output.backward()
    
    print(weights.grad)

    # optimize model, i.e. adjust weights...
    with torch.no_grad():
        weights -= 0.1 * weights.grad

    # this is important! It affects the final weights & output
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [46]:
print(weights)
print(model_output)

tensor([0.1000, 0.1000, 0.1000, 0.1000], requires_grad=True)
tensor(4.8000, grad_fn=<SumBackward0>)


- Optimizer has zero_grad() method
- optimizer = torch.optim.SGD([weights], lr=0.1)
##### During training:
- optimizer.step()
- optimizer.zero_grad()